In [1]:
# Bibliotecas externas
import pyodbc
from datetime import datetime
import win32com.client as win32
from datetime import datetime, timedelta



      # String de conexão
connection_string = (
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER=172.17.1.115;"
    f"DATABASE=dbdatastoreHouse;"
    f"UID=USR_MIS;"
    f"PWD=MIS@321"
)


def EncaminhaEmails(CodigoCarteira,ParaEnvioDaily,ParaEnvioMis):

    conn = pyodbc.connect(connection_string)
    cursor = conn.cursor()

    query_operacao = """execute geral.EmailDailyToPy ?, ?"""

    for row in ParaEnvioDaily:    
        cursor.execute(query_operacao, CodigoCarteira,1)
    for row in ParaEnvioMis:    
        cursor.execute(query_operacao, CodigoCarteira,0)
        
    cursor.close()
    conn.close()


def AtualizaMacro(CodigoCarteira, Banco, Carteira, Filial):
    # Data de hoje
    data_hoje = datetime.now()

    # Primeira data do mês
    primeira_data_do_mes = data_hoje.replace(day=1)

    # Formatar as datas para o formato aaaa-mm-dd
    data_hoje_formatada = data_hoje.strftime("%Y-%m-%d")
    primeira_data_do_mes_formatada = primeira_data_do_mes.strftime("%Y-%m-%d")
    
    arrayParaVisualizarMelhor = [CodigoCarteira, Banco, Carteira, Filial]
    
    # Caminho do arquivo Excel
    caminho_excel = rf"\\172.17.1.115\TEMP\\DailysToPy\Dailys\{Filial}\Daily_{CodigoCarteira}_{Banco}_{Carteira}.xlsb"

    excel = None
    try:
        # Inicia uma instância do Excel
        excel = win32.Dispatch("Excel.Application")
        excel.Visible = False

        # Abre o arquivo Excel
        workbook = excel.Workbooks.Open(Filename=caminho_excel)
        
        # Habilita as macros
        excel.AutomationSecurity = 1  # Habilita todas as macros
        
        # Atualiza as células
        sheet = workbook.Sheets('Aux_SQL')
        sheet.Range("G2").Value = primeira_data_do_mes_formatada
        sheet.Range("H2").Value = data_hoje_formatada
        
        # Executa a macro
        excel.Application.Run('Atualiza_Tudo.Atu_tudo')
        print(f"Macro executada com sucesso para {arrayParaVisualizarMelhor}!")
        
        # Salva e fecha o arquivo
        workbook.Save()
        workbook.Close(SaveChanges=1)

    except Exception as e:
        print(f"Erro ao executar a macro para {arrayParaVisualizarMelhor}: {e}")
        excel.Application.Quit()
        del excel
    finally:
        # Garantir que o Excel seja encerrado corretamente
        if excel is not None:
            excel.Application.Quit()
            del excel
        print(f"Processo concluído para {arrayParaVisualizarMelhor}.")


def ConsultaValida_ToAtualizaMacro():
    # Data de hoje
    data_hoje = datetime.now()

    # Formatar as datas para o formato aaaa-mm-dd
    data_hoje_formatada = data_hoje.strftime("%Y-%m-%d")
    # Tentar se conectar e processar
    try:
        conn = pyodbc.connect(connection_string)
        cursor = conn.cursor()

        # Consulta SQL
        query = """  select	
                        a.CodigoCarteira,
                        b.Banco,
                        b.Carteira,
                        b.Filial,
                        a.msg
                        
                    from dbdatastorehouse.dbo.MapaDailyPesoToPy a with(nolock)
                    join dbdatastorehouse.dbo.dp_dailytopy b on a.CodigoCarteira = b.GrupoCliente
                    where convert(date,a.DataAtualizacao) = ? and CodigoCarteira = 911
                """

        # Executar a consulta e armazenar os resultados
        cursor.execute(query, data_hoje_formatada)
        arrayCdCli = cursor.fetchall()
        print(arrayCdCli)
        # Fechar conexão
        cursor.close()
        conn.close()

        #Para envio e para não envio
        vetor_envio = []
        vetor_nao_envio = []
        if arrayCdCli:
            for row in arrayCdCli:
                CodigoCarteira, Banco, Carteira, Filial, Msg = row
                if Msg == "Enviar":
                    vetor_envio.append(row)
                elif Msg == "NÃO enviar":
                    vetor_nao_envio.append(row)

                # Tentar rodar a macro para cada resultado
                try:
                    AtualizaMacro(CodigoCarteira, Banco, Carteira, Filial)
                    EncaminhaEmails(CodigoCarteira,vetor_envio,vetor_nao_envio)
                except Exception as e:
                    print(f"Erro em: {row}: {e}")
        else:
            print('Nenhum resultado encontrado.')
    except Exception as e:
        print(f"Erro ao conectar ou executar a consulta: {e}")

ConsultaValida_ToAtualizaMacro();




[(911, 'Bradesco', 'COMERCIAL', 'Marechal', 'Enviar'), (911, 'Bradesco', 'COMERCIAL', 'Marechal', 'Enviar')]
Erro ao executar a macro para [911, 'Bradesco', 'COMERCIAL', 'Marechal']: Property 'Excel.Application.Visible' can not be set.
Erro em: (911, 'Bradesco', 'COMERCIAL', 'Marechal', 'Enviar'): Excel.Application.Application


In [7]:
# Bibliotecas externas
import pyodbc

# String de conexão
connection_string = (
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER=172.17.1.115;"
    f"DATABASE=dbdatastoreHouse;"
    f"UID=USR_MIS;"
    f"PWD=MIS@321"
)


def EncaminhaEmails(CodigoCarteira, ValidaEnvio):
    conn = pyodbc.connect(connection_string)
    cursor = conn.cursor()

    query_operacao = """execute geral.EmailDailyToPy ?, ?"""
    cursor.execute(query_operacao, CodigoCarteira, ValidaEnvio)
    
    # try:
    #     # Tenta executar a operação
    #     cursor.execute(query_operacao, CodigoCarteira, ValidaEnvio)
        
    #     # Verifica se a execução afetou alguma linha
    #     if cursor.rowcount > 0:
    #         print(f"Envio de e-mails bem-sucedido para Código da Carteira: {CodigoCarteira}. Linhas afetadas: {cursor.rowcount}.")
    #     else:
    #         print(f"Nenhum e-mail enviado para Código da Carteira: {CodigoCarteira}.")
    
    # except Exception as e:
    #     print(f"Erro ao enviar e-mails: {e}")
    
    # finally:
    cursor.close()
    conn.close()

# Exemplo de uso
EncaminhaEmails(911, 0)  # Altere os parâmetros conforme necessário
